In [2]:
import torch
import gpytorch
import numpy as np
import pandas as pd
import os
from tqdm import tqdm

# Prepare Data

In [3]:
data_dir = "../data/Jan1524_Jan2224/"
data_files = [file for file in os.listdir(data_dir) if file.endswith(".csv")]
data = []
for file in data_files:
    df = pd.read_csv(data_dir + file, index_col=0)
    # decompose timestamp
    df["timestamp"] = pd.to_datetime(df["timestamp"],format="mixed")
    df["hour"] = df["timestamp"].dt.hour
    df["day"] = df["timestamp"].dt.day
    df["month"] = df["timestamp"].dt.month
    df["weekday"] = df["timestamp"].dt.weekday
    
    df = df.loc[:, ["longitude", "latitude", "celsius", "humidity", "pressure", "month", "day", "weekday", "hour", "pm25"]]
    df = df.groupby(["month", "day", "hour", "weekday"]).mean().reset_index(drop=False)
    
    df.loc[df["pm25"] < 0, "pm25"] = 0
    if len(df) < 7 * 24:
        continue
    else:
        data.append(df)
data = np.array(data).transpose(1, 0, 2)
X = data[:, :, :-1]
Y = data[:, :, -1]
print(X.shape, Y.shape)

(168, 8, 9) (168, 8)


# Construct GP model

In [4]:
class LocalPeriodicKernel(gpytorch.kernels.Kernel):
    is_stationary = True

    def __init__(self, lp_ard=None, **kwargs):
        super().__init__(**kwargs)
        if lp_ard is not None:
            self.periodickernel = gpytorch.kernels.PeriodicKernel(arg_num_dims=lp_ard)
            self.rbfkernel = gpytorch.kernels.RBFKernel(arg_num_dims=lp_ard)
        else:
            self.periodickernel = gpytorch.kernels.PeriodicKernel()
            self.rbfkernel = gpytorch.kernels.RBFKernel()
        self.localperiodickernel = self.periodickernel * self.rbfkernel

    #kernel function
    def forward(self, x1, x2, **params):
        return self.localperiodickernel(x1, x2, **params)
    
class BaseKernel(gpytorch.kernels.Kernel):
    def __init__(self, matern_ard=None, lp_ard=None, **kwargs):
        super().__init__(**kwargs)
        if matern_ard is not None:
            self.maternkernel = gpytorch.kernels.MaternKernel(nu=0.5,ard_num_dims=matern_ard)
        else:
            self.maternkernel = gpytorch.kernels.MaternKernel(nu=0.5)
        if lp_ard is not None:
            self.localperiodickernel = LocalPeriodicKernel(lp_ard=lp_ard)
        else:
            self.localperiodickernel = LocalPeriodicKernel()

    def forward(self, x1, x2, **params):
        # separate the input into continuous and periodic components
        x1_per = x1[:, :4]
        x1_cont = x1[:, 4:]
        x2_per = x2[:, :4]
        x2_cont = x2[:, 4:]
        return self.maternkernel(x1_cont, x2_cont, **params) * self.localperiodickernel(x1_per, x2_per, **params)

class GlobalKernel(gpytorch.kernels.Kernel):
    is_stationary = True

    def __init__(self, matern_ard=None, lp_ard=None, **kwargs):
        super().__init__(**kwargs)

        # base kernel
        self.basekernel = BaseKernel(matern_ard=matern_ard, lp_ard=lp_ard)

        # scale kernel
        self.scalekernel = gpytorch.kernels.ScaleKernel(self.basekernel)

    
    def forward(self, x1, x2, **params):
        return self.scalekernel(x1, x2, **params)

In [5]:
class AirGP(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood, matern_ard=None, lp_ard=None):
        super(AirGP, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = GlobalKernel(matern_ard=matern_ard, lp_ard=lp_ard)

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

# LOOCV

In [6]:
n_steps, n_stations = X.shape[:2]

In [7]:
RMSEs = []
for t in tqdm(np.random.permutation(n_steps)[:120]):
    for i in range(n_stations):
            # Leave-one-out split
            X_train = torch.from_numpy(np.concatenate((X[t, :i], X[t, i+1:]), axis=0))
            X_test = torch.from_numpy(X[t, i:i+1])
            Y_train = torch.from_numpy(np.concatenate((Y[t, :i], Y[t, i+1:]), axis=0))
            Y_test = torch.from_numpy(Y[t, i:i+1])
            # prepare training
            likelihood = gpytorch.likelihoods.GaussianLikelihood()
            model = AirGP(X_train, Y_train, likelihood)
            training_iter = 10000
            model.train()
            likelihood.train()
            optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
            mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)
            # training
            for iter in range(training_iter):
                model.train()
                likelihood.train()
                optimizer.zero_grad()
                output = model(X_train)
                loss = -mll(output, Y_train)
                loss.backward()
                optimizer.step()

            # evaluation
            model.eval()
            likelihood.eval()
            Y_pred = model(X_test)
            Y_pred_mean = Y_pred.mean.detach()
            rmse = torch.sqrt(torch.mean((Y_pred_mean - Y_test) ** 2))
            RMSEs.append(rmse)

  0%|          | 0/120 [00:00<?, ?it/s]

  4%|▍         | 5/120 [12:25<4:45:42, 149.06s/it]


KeyboardInterrupt: 

In [9]:
np.mean(RMSEs)

1.6662676658445068